In [2]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [3]:
llm = Ollama(model="mistral",  callbacks=CallbackManager([StreamingStdOutCallbackHandler()]),num_gpu=1, base_url="http://localhost:11434")

In [5]:
llm("Hello world")

 Hello! It's nice to meet you. I'm an assistant designed to help answer questions and provide information. Is there something specific you would like to know or ask about? I'm here to help. If you just wanted to say hello, then hey there! How was your day? Let me know if you have any questions or need assistance with anything else.

" Hello! It's nice to meet you. I'm an assistant designed to help answer questions and provide information. Is there something specific you would like to know or ask about? I'm here to help. If you just wanted to say hello, then hey there! How was your day? Let me know if you have any questions or need assistance with anything else."

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings


modelPath = "BAAI/bge-small-en"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cuda:0'}
encode_kwargs = {'normalize_embeddings': True}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embedding = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

/home/sid/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

# loader = DirectoryLoader('mytext', glob='*.txt', loader_cls=TextLoader)
# documents = loader.load()


In [23]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data.pdf")
documents = loader.load_and_split()

In [24]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [25]:
len(texts)
import torch
torch.cuda.empty_cache()
# print(torch.cuda.memory_summary(device=None, abbreviated=False))
import gc
torch.cuda.empty_cache()
gc.collect()

1276

In [26]:
from langchain.vectorstores import Chroma
persist_directory = './db'
vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)
vectordb.persist()

In [27]:
import torch
dev = "cuda:0" if torch.cuda.is_available() else "cpu"
dev

'cuda:0'

In [40]:
retriever = vectordb.as_retriever(search_kwargs={'k':5})

In [41]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm=llm, 
                                  retriever=retriever, 
                                  return_source_documents=True)
     

In [60]:
def process_llm_response(query):
    llm_response = qa_chain(query)
    return llm_response['result']
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [66]:
query = "How to do grade improvement? "
process_llm_response(query)

 To do grade improvement, you should follow these steps:
1. Identify the courses in which you would like to improve your grade.
2. Speak with your professor during their office hours to discuss any areas where you may need additional help or clarification on course material.
3. Utilize campus resources such as tutoring services,

KeyboardInterrupt: 

In [67]:
def main():
    st.title("VIT QA System")
    user_input = st.text_area("Enter Query")

    if st.button("Get response"):
        result = process_llm_response(user_input)
        st.write("Response:")
        # make text bigger
        st.write(f"**{result}**")
main()